# Upload data to S3

In [37]:
import sagemaker
import pathlib
import boto3

In [38]:
BASE_DIR = pathlib.Path().resolve().parent

DATA_DIR = BASE_DIR / 'data'
MODEL_DIR = BASE_DIR / 'models'

MODEL_DIR.mkdir(parents=True, exist_ok=True)

EXPORTS_DIR = DATA_DIR / 'exports'
XRAY_LUNG_CLF_DIR = DATA_DIR / 'xray_lung_clf'

TRAIN_DATA_PATH = XRAY_LUNG_CLF_DIR / 'train'
TEST_DATA_PATH = XRAY_LUNG_CLF_DIR / 'test'

TRAIN_ANNOTATIONS_PATH = XRAY_LUNG_CLF_DIR / 'train_annotations.json'
TEST_ANNOTATIONS_PATH = XRAY_LUNG_CLF_DIR / 'test_annotations.json'

EXPORTS_LUNG_CLF_DIR = EXPORTS_DIR / 'xray_lung_clf'

serve_pytorch_dir = BASE_DIR / 'serve_pytorch'
train_py_path = serve_pytorch_dir / 'train.py'

In [39]:
# Create session
session = sagemaker.Session()

# Get Role
role = sagemaker.get_execution_role()

# Get Bucket
bucket = boto3.resource('s3').Bucket('lung-xray-bucket')

In [19]:
# Get data dir
#data_dir = str(EXPORTS_LUNG_CLF_DIR)

# set prefix, a descriptive name for a directory  
#prefix = 'lung_data'

# upload all data to S3
#input_data = session.upload_data(data_dir, bucket=bucket, key_prefix=prefix)
#print(input_data)

s3://sagemaker-us-east-1-906764816799/lung_data


In [40]:
for obj in bucket.objects.all():
    print(obj.key)

data/
data/test_X.z
data/test_Y.z
data/train_X.z
data/train_Y.z


# Create PyTorch esitmator

Debugging learned:
- need to install some packages on remote EC2 for training job -> put requirements.txt in same folder as train.py and model.py
- data needs to be stored in same region as sagemaker notebook instance
- creating a notebook instance in one region means it will not be visible from another
- the blue printing message lists al the SM_* enviroment variables, when prompted with a KeyError, check if key is present. Example: Needed to change SM_CHANNEL_TRAIN to SM_CHANNEL_TRAINING.
- valid loss increase after 84 epochs
- BLEWITHLOGISTLOSS combines sigmoid and BCE in one class

In [41]:
prefix = 'data'

bucket_name = str(bucket.name)

input_data = f's3://{bucket_name}/{prefix}'

print(f'Downloading data from source: {input_data}')

In [47]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
                    entry_point = 'train.py',
                    source_dir = str(serve_pytorch_dir),
                    role = role,
                    instance_count = 1,
                    instance_type = 'ml.c5.xlarge',
                    framework_version='1.8.0',
                    py_version='py3',
                    sagemaker_session = session)

estimator.fit(input_data)

2021-12-10 12:56:27 Starting - Starting the training job...
2021-12-10 12:56:29 Starting - Launching requested ML instancesProfilerReport-1639140987: InProgress
......
2021-12-10 12:57:56 Starting - Preparing the instances for training.........
2021-12-10 12:59:20 Downloading - Downloading input data
2021-12-10 12:59:20 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-12-10 12:59:34,309 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-12-10 12:59:34,311 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-12-10 12:59:34,319 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-12-10 12:59:34,939 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-12-10 12:59:35,195 sagemaker-training-toolkit INFO     Installing dependencies fro

UnexpectedStatusException: Error for Training job pytorch-training-2021-12-10-12-56-27-587: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/opt/conda/bin/python3.6 train.py"
Traceback (most recent call last):
  File "train.py", line 307, in <module>
    model_fn(model_dir)
NameError: name 'model_dir' is not defined

# Deploy Estimator